# ================================================
# PIPELINE DE MACHINE LEARNING COM PLN E VECTORDB
# ================================================

## INSTALAÇÃO DE DEPENDÊNCIAS

In [ ]:
# !pip install sentence-transformers
# !pip install chromadb
# !pip install pdfplumber
# !pip install nltk
# !pip install spacy
# !python -m spacy download pt_core_news_sm

## IMPORTS

In [112]:
import json
import csv
import re
import numpy as np
import pickle
import os
import spacy
import nltk
from nltk.corpus import stopwords
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_text_splitters import CharacterTextSplitter
from langchain_text_splitters import MarkdownHeaderTextSplitter

#nltk.download('stopwords') # rodar apenas uma vez

## ETAPAS

### 1. CARREGAMENTO DE ARQUIVOS

#### Definição das funções

In [7]:
def carregar_pdf(caminho_arquivo):
    """Carrega e extrai texto de um arquivo PDF"""
    try:
        import pdfplumber
        texto_completo = []
        with pdfplumber.open(caminho_arquivo) as pdf:
            for pagina in pdf.pages:
                texto = pagina.extract_text()
                if texto:
                    texto_completo.append(texto)
        return "\n".join(texto_completo)
    except ImportError:
        print("⚠ pdfplumber não disponível. Instale com: pip install pdfplumber")
        return ""

def carregar_csv(caminho_arquivo):
    """Carrega e extrai texto de um arquivo CSV"""
    textos = []
    with open(caminho_arquivo, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for linha in reader:
            texto = " ".join([str(v) for v in linha.values()])
            textos.append(texto)
    return "\n".join(textos)

def carregar_json(caminho_arquivo):
    """Carrega e extrai texto de um arquivo JSON"""
    with open(caminho_arquivo, 'r', encoding='utf-8') as f:
        dados = json.load(f)

    textos = []

    def extrair_texto_recursivo(obj):
        if isinstance(obj, dict):
            for valor in obj.values():
                extrair_texto_recursivo(valor)
        elif isinstance(obj, list):
            for item in obj:
                extrair_texto_recursivo(item)
        elif isinstance(obj, str):
            textos.append(obj)

    extrair_texto_recursivo(dados)
    return "\n".join(textos)

def carregar_txt(caminho_arquivo):
    """Carrega e extrai texto de um arquivo TXT"""
    with open(caminho_arquivo, 'r', encoding='utf-8') as f:
        return f.read()

def carregar_arquivo(caminho_arquivo):
    """Função universal para carregar arquivos PDF, CSV ou JSON"""
    extensao = caminho_arquivo.split('.')[-1].lower()

    if extensao == 'pdf':
        return carregar_pdf(caminho_arquivo)
    elif extensao == 'csv':
        return carregar_csv(caminho_arquivo)
    elif extensao == 'json':
        return carregar_json(caminho_arquivo)
    elif extensao == 'txt':
        return carregar_txt(caminho_arquivo)
    else:
        raise ValueError(f"Formato não suportado: {extensao}")

#### Execução

In [11]:
texto = ""
texto = carregar_arquivo('introducaoml.pdf')
print(texto)

Título: Introdução ao Machine Learning
Machine learning é um campo da inteligência artificial que desenvolve algoritmos
capazes de aprender padrões a partir de dados. Os principais tipos incluem
aprendizado supervisionado, aprendizado não supervisionado e aprendizado por
reforço.
Aprendizado supervisionado envolve treinar um modelo com dados rotulados,
como prever preços de casas com base em características como tamanho,
localização e número de quartos. Aprendizado não supervisionado detecta
padrões ocultos em dados não rotulados, como segmentação de clientes em
marketing. Aprendizado por reforço ensina agentes a tomar decisões em
ambientes dinâmicos para maximizar recompensas.
Redes neurais profundas são usadas em visão computacional e processamento
de linguagem natural. Modelos como CNNs e Transformers têm revolucionado
tarefas como tradução automática, reconhecimento de imagens e geração de
texto.


### 2. PROCESSAMENTO DE LINGUAGEM NATURAL (PLN)

#### Definição das Stop Words

In [41]:
nlp = spacy.load('pt_core_news_sm')

API_STOPWORDS_PT = set(stopwords.words('portuguese'))
API_STOPWORDS_PT.remove('não')  # Mantendo a palavra "não"

STOPWORDS_EN = {'a', 'an', 'and', 'are', 'as', 'at', 'be', 'by', 'for', 'from', 
                'has', 'he', 'in', 'is', 'it', 'its', 'of', 'on', 'that', 'the', 
                'to', 'was', 'will', 'with', 'this', 'but', 'they', 'have', 'had'}

# Carregando apenas as stopwords em português

STOPWORDS_ALL = API_STOPWORDS_PT

#### Pré-Processamento de Texto

In [105]:
def limpar_texto(texto):
    """Realiza limpeza básica do texto"""
    texto = texto.lower()
    texto = re.sub(r'\n\n', ' \n\n', texto) # manter quebras de parágrafo em caso de recursive chunking
    texto = re.sub(r'\n', ' \n', texto)
    texto = re.sub(r'\.', ' .', texto)
    texto = re.sub(r'[^a-z0-9áàâãéêíóôõúüçñ\s\n.]', ' ', texto) # ^ indica negação
    texto = re.sub(r'  +', ' ', texto) # multiplos espaços
    return texto.strip()

# Substituir acentos em caso de stemização

def tokenizar_simples(texto):
    """Tokeniza o texto (divisão por espaços)"""
    return texto.split()

def tokenizar_spacy(texto):
    """Tokeniza o texto usando spaCy"""
    doc = nlp(texto)
    return [token for token in doc]

def remover_stopwords(tokens, stopwords=STOPWORDS_ALL):
    """Remove stopwords em português e inglês"""
    return [token for token in tokens if token.text not in stopwords]

def lematizar_texto(tokens):
    """Pega a palavra lema de cada token no texto"""
    return [token.lemma_ for token in tokens]

def processar_texto_pln(texto):
    """Pipeline completo de PLN"""
    texto_limpo = limpar_texto(texto)
    palavras = tokenizar_spacy(texto_limpo)
    palavras_sem_stopwords = remover_stopwords(palavras)
    palavras_lemma = lematizar_texto(palavras_sem_stopwords)
    return ' '.join(palavras_lemma)

#### Execução

In [106]:
texto_processado = processar_texto_pln(texto)
print(texto_processado)

título introdução machine learning 
 machine learning campo inteligência artificial desenvolver algorit 
 capaz aprender padrão partir dado . principal tipo incluir 
 aprendizar supervisionar aprendizar não supervisionar aprendizar 
 reforço . 
 aprendizar supervisionar envolver treinar modelo dado rotular 
 prever preço casa base característica tamanho 
 localização número quarto . aprendizar não supervisionar detectar 
 padrão oculto dado não rotular segmentação cliente 
 marketing . aprendizar reforço ensina agente tomar decisão 
 ambiente dinâmico maximizar recompensa . 
 rede neural profundo usar visão computacional processamento 
 linguagem natural . modelo cnns transformer ter revolucionar 
 tarefa tradução automático reconhecimento imagem geração 
 texto .


### 3. CHUNKING COM OVERLAP

#### Tipos relevantes de chunking

##### Fixed-Length Chunking

Divide o texto em pedaços com número fixo de caracteres ou tokens. É simples de implementar e previsível, porém pode cortar informações importantes.

In [111]:
# EXEMPLO:

fixed_splitter = CharacterTextSplitter(
    separator=".",
    chunk_size=150,
    chunk_overlap=20
)

chunks = fixed_splitter.split_text(texto_processado)
print("n_chunks: ", len(chunks))
print(chunks)

n_chunks:  7
['título introdução machine learning \n machine learning campo inteligência artificial desenvolver algorit \n capaz aprender padrão partir dado', 'principal tipo incluir \n aprendizar supervisionar aprendizar não supervisionar aprendizar \n reforço', 'aprendizar supervisionar envolver treinar modelo dado rotular \n prever preço casa base característica tamanho \n localização número quarto', 'aprendizar não supervisionar detectar \n padrão oculto dado não rotular segmentação cliente \n marketing', 'aprendizar reforço ensina agente tomar decisão \n ambiente dinâmico maximizar recompensa', 'rede neural profundo usar visão computacional processamento \n linguagem natural', 'modelo cnns transformer ter revolucionar \n tarefa tradução automático reconhecimento imagem geração \n texto']


##### Recursive Character Chunking

Divide o texto progressivamente, mantendo mantendo estruturas naturais (parágrafos, frases), a melhor abordagem na maioria dos casos, superior ao fixed-length

In [113]:
# EXEMPLO:

recursive_splitter = RecursiveCharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=20,
    separators=["\n\n", ".", "\n", " ", ""] # '.' tem mais valor semântico que '\n' pelo modo como o arquivo(pdf) é carregado
)

chunks = recursive_splitter.split_text(texto_processado)
print("n_chunks: ", len(chunks))
print(chunks)

n_chunks:  7
['título introdução machine learning \n machine learning campo inteligência artificial desenvolver algorit \n capaz aprender padrão partir dado', '. principal tipo incluir \n aprendizar supervisionar aprendizar não supervisionar aprendizar \n reforço', '. \n aprendizar supervisionar envolver treinar modelo dado rotular \n prever preço casa base característica tamanho \n localização número quarto', '. aprendizar não supervisionar detectar \n padrão oculto dado não rotular segmentação cliente \n marketing', '. aprendizar reforço ensina agente tomar decisão \n ambiente dinâmico maximizar recompensa', '. \n rede neural profundo usar visão computacional processamento \n linguagem natural', '. modelo cnns transformer ter revolucionar \n tarefa tradução automático reconhecimento imagem geração \n texto .']


##### Document-Based Chunking

Divide o texto respeitando a estrutura hierárquica do documento, ótima abordagem para casos envolvendo textos com markdown, artigos científicos e acadêmicos.

In [ ]:
# EXEMPLO COM MARKDOWN:

headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on
)

chunks_document = markdown_splitter.split_text(documento_markdown)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=20
)

final_chunks = text_splitter.split_documents(chunks_document)

# chunks_document
# metadata definidos no headers_to_split_on -> chunk.metadata # {'Header 1': 'Introdução ao PLN'}
# conteúdo  # Texto ...

### 4. EMBEDDING

In [7]:
def criar_embeddings(chunks, usar_modelo_real=True):
    """Cria embeddings para os chunks"""
    if usar_modelo_real:
        try:
            from sentence_transformers import SentenceTransformer

            print("Carregando modelo de embedding...")
            modelo = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
            print("✓ Modelo carregado")

            embeddings = modelo.encode(chunks, show_progress_bar=True)
            return embeddings

        except ImportError:
            print("⚠ sentence-transformers não disponível")
            print("Usando embeddings simulados...")
            usar_modelo_real = False

    if not usar_modelo_real:
        # Embeddings simulados para demonstração
        dimensao = 384
        embeddings = np.random.randn(len(chunks), dimensao).astype('float32')
        return embeddings

### 5. VECTOR DATABASE

In [8]:
class VectorDB:
    """Banco de dados vetorial simples"""

    def __init__(self):
        self.embeddings = []
        self.documentos = []
        self.metadados = []
        self.ids = []

    def adicionar(self, embeddings, documentos, metadados, ids):
        """Adiciona documentos ao banco"""
        self.embeddings = np.array(embeddings)
        self.documentos = documentos
        self.metadados = metadados
        self.ids = ids

    def buscar(self, query_embedding, n_resultados=3):
        """Busca documentos similares (similaridade de cosseno)"""
        query_norm = query_embedding / np.linalg.norm(query_embedding)
        docs_norm = self.embeddings / np.linalg.norm(self.embeddings, axis=1, keepdims=True)

        similaridades = np.dot(docs_norm, query_norm)
        indices_top = np.argsort(similaridades)[::-1][:n_resultados]

        resultados = []
        for idx in indices_top:
            resultados.append({
                'id': self.ids[idx],
                'documento': self.documentos[idx],
                'metadados': self.metadados[idx],
                'similaridade': float(similaridades[idx])
            })

        return resultados

    def salvar(self, caminho='vectordb.pkl'):
        """Salva o banco em arquivo"""
        dados = {
            'embeddings': self.embeddings,
            'documentos': self.documentos,
            'metadados': self.metadados,
            'ids': self.ids
        }
        with open(caminho, 'wb') as f:
            pickle.dump(dados, f)
        print(f"✓ Banco salvo em: {caminho}")

    def carregar(self, caminho='vectordb.pkl'):
        """Carrega o banco de arquivo"""
        with open(caminho, 'rb') as f:
            dados = pickle.load(f)
        self.embeddings = dados['embeddings']
        self.documentos = dados['documentos']
        self.metadados = dados['metadados']
        self.ids = dados['ids']
        print(f"✓ Banco carregado de: {caminho}")

    def contar(self):
        """Retorna número de documentos"""
        return len(self.documentos)

### 6. BUSCA VETORIAL (QUERY)

In [9]:
def realizar_busca_vetorial(vector_db, query, processar_query=True, n_resultados=3):
    """
    Realiza busca vetorial

    Args:
        vector_db: Instância do banco de dados vetorial
        query: Texto da query
        processar_query: Se True, aplica PLN na query
        n_resultados: Número de resultados a retornar
    """
    print(f"\nQUERY: {query}")
    print("-" * 70)

    # Processar query (mesmo pipeline do texto)
    if processar_query:
        query_processada = processar_texto_pln(query)
        print(f"Query processada: {query_processada}")
    else:
        query_processada = query

    # Criar embedding da query
    # Nota: Em produção, usar o mesmo modelo dos documentos
    query_embedding = criar_embeddings([query_processada], usar_modelo_real=False)[0]

    # Buscar documentos similares
    resultados = vector_db.buscar(query_embedding, n_resultados=n_resultados)

    print(f"\nResultados: {len(resultados)}")
    for i, resultado in enumerate(resultados, 1):
        print(f"\n[{i}] ID: {resultado['id']}")
        print(f"    Similaridade: {resultado['similaridade']:.4f}")
        print(f"    Documento: {resultado['documento'][:150]}...")

    return resultados

In [13]:
print("\n" + "="*80)
print("EXEMPLO DE USO DO PIPELINE")
print("="*80)

# 1. Carregar arquivos
texto = carregar_arquivo('indroducaoml.pdf')
#texto = "Seu texto de exemplo aqui..."

# 2. Processar com PLN
texto_processado = processar_texto_pln(texto)
print(f"\n✓ Texto processado: {len(texto_processado)} caracteres")

# 3. Criar chunks
chunks = criar_chunks(texto_processado, tamanho_chunk=50, overlap=5)
print(f"✓ Criados {len(chunks)} chunks")

# 4. Criar embeddings
embeddings = criar_embeddings(chunks, usar_modelo_real=False)
print(f"✓ Embeddings criados: {embeddings.shape}")

# 5. Criar e popular banco vetorial
vector_db = VectorDB()
ids = [f"chunk_{i}" for i in range(len(chunks))]
metadados = [{"index": i} for i in range(len(chunks))]
vector_db.adicionar(embeddings, chunks, metadados, ids)
print(f"✓ Banco vetorial criado com {vector_db.contar()} documentos")

# 6. Salvar banco
vector_db.salvar('meu_vectordb.pkl')

# 7. Realizar buscas
queries = [
    "machine learning",
    "processamento de linguagem natural",
    "inteligência artificial"
]

for query in queries:
    realizar_busca_vetorial(vector_db, query, n_resultados=2)

print("\n" + "="*80)
print("PIPELINE CONCLUÍDO COM SUCESSO!")
print("="*80)


EXEMPLO DE USO DO PIPELINE

✓ Texto processado: 433 caracteres
✓ Criados 1 chunks
✓ Embeddings criados: (1, 384)
✓ Banco vetorial criado com 1 documentos
✓ Banco salvo em: meu_vectordb.pkl

QUERY: machine learning
----------------------------------------------------------------------
Query processada: machine learning

Resultados: 1

[1] ID: chunk_0
    Similaridade: -0.0554
    Documento: documento teste machine learning embeddings representacao vetori embeddings sao representacoes numericas dados espaco vetori alta dimensao word embedd...

QUERY: processamento de linguagem natural
----------------------------------------------------------------------
Query processada: processamento linguagem natur

Resultados: 1

[1] ID: chunk_0
    Similaridade: -0.0262
    Documento: documento teste machine learning embeddings representacao vetori embeddings sao representacoes numericas dados espaco vetori alta dimensao word embedd...

QUERY: inteligência artificial
-------------------------------